In [15]:
# Imports 

import pandas as pd 
import numpy as np
import os 

In [3]:
data = pd.read_csv('../data/data_reduced.csv')
raw_data = pd.read_csv('../data/data_notscaled.csv')
data

,Factor1,Factor2,Factor3,Factor4
0,-1.097881,3.042659,0.441110,1.794368
1,0.837712,5.054604,4.593511,1.305088
2,-2.678498,5.816443,-3.505566,-0.007615
3,14.018654,1.812564,3.009246,-6.586102
4,-9.822322,-3.651578,0.906058,-2.784624
...,...,...,...,...
2759,0.087177,0.925191,-1.526079,-3.750073
2760,-3.611693,2.389590,7.200176,-3.914275
2761,-4.682466,-2.267832,-0.798776,2.611891
2762,1.248353,-2.739698,-2.559083,-5.045817


I want to give the user the option to see recommended cats that are most or least like their own cat in each trait. For example, someone may want a cat that is similar to their current cat in terms of dominance and intelligence, but the opposite in terms of activity level. In order to do so I will define a custom distance. For each trait, the distance must be minimized if the user selected to see recommendations that are the most similar in that trait, and maximized if they wish to see the opposite. 

In [4]:
traits = data.columns.tolist()

In [47]:
trying = np.array(test) - np.array(test2)

In [ ]:
# Defining the maxmimun distance squared for each trait

max_dists_sq = (data.max() - data.min()) ** 2

In [101]:
def distance(user1, user2, same_traits=None, different_traits=None):

    user1 = np.array(user1, dtype=float).flatten()
    user2 = np.array(user2, dtype=float).flatten()
    diff = user1 - user2 
    dist = 0.0
    n = len(user1)

    # if no traits are specified to be similar or different, the default is to recommend based on similarities in all traits 
    if same_traits is None and different_traits is None:
        same_traits = range(n)
        different_traits = []

    elif same_traits is None:
        same_traits = [i for i in range(n) if i not in different_traits]
        
    elif different_traits is None:
        different_traits = [i for i in range(n) if i not in same_traits]
        
    for i in same_traits:
        dist += diff[i] ** 2

    for i in different_traits:
        max_val = max_dists_sq.iloc[i]
        dist += (max_val - diff[i] ** 2)

    return dist

In [102]:
def recommend(new_user, data, same_traits=None, different_traits=None, n_recs=10, factors=None):

    new_user = np.array(new_user, dtype=float)
    distances = []
    if factors is None:
        factors=data.columns.tolist()

    for i, row in data.iterrows(): 
        user2 = np.array(row[factors], dtype=float)

        d = distance(new_user, user2, same_traits, different_traits)
        distances.append((i, d))
    
    # Sort by ascending distance (best matches first)
    distances.sort(key=lambda x: x[1])
    
    return distances[:n_recs]
    

In [84]:
recommend(test, data, same_traits, diff_traits, n_recs=3)

[(507, 952.6126958737867), (601, 953.7503478744593), (154, 966.4527494838017)]

In [86]:
recommend(test2, data, different_traits=[2])

[(2209, 293.90414086439074),
 (2309, 307.8656977938675),
 (2499, 310.7409865516853),
 (1147, 338.31722008322333),
 (1911, 340.8500860084431),
 (547, 343.5479729338443),
 (571, 348.14735703658994),
 (877, 352.30603096949255),
 (2760, 354.62713165479283),
 (2304, 356.74200964041484)]

In [103]:
recommend(test4, data)

[(1377, 0.5653610501056178),
 (1742, 0.6253745217185432),
 (1733, 0.6746091782413869),
 (2370, 1.049443787696961),
 (1535, 1.5042388336411647),
 (2199, 1.8389207553460123),
 (1783, 2.2837995509971996),
 (1056, 2.365998475544991),
 (478, 2.735586405991848),
 (1650, 3.021892567706973)]

In [106]:
raw_data.iloc[1377]

Vigilant                 5
Stable                   6
Bold                     4
Clumsy                   2
Defiant                  4
Gentle                   4
Constrained              5
Inquisitive              4
Inventive                6
Irritable                2
Distractable             3
Erratic                  3
Solitary                 3
Impulsive                4
Quitting                 3
Independent              5
Smart                    6
Jealous                  6
Fearful_other_cats       3
Persevering              5
Greedy                   3
Friendly_other_cats      4
Submissive               4
Dominant                 4
Reckless                 3
Predictable              6
Suspicious               5
Individualistic          6
Affectionate             6
Insecure                 3
Bullying                 5
Curious                  4
Aimless                  2
Deliberate               6
Tense                    3
Fearful_of_people        3
Cool                     5
A

In [105]:
test4

array([[-0.78446454],
       [ 0.78446454],
       [ 1.17669681],
       [-1.17669681]])

In [92]:
from sklearn.preprocessing import StandardScaler

test3 = np.array([2, 6, 7, 1])

scaler = StandardScaler()
test4 = scaler.fit_transform(test3.reshape(-1, 1))

In [113]:
data = data.round(5)
data.describe()

,Factor1,Factor2,Factor3,Factor4
count,2.764000e+03,2.764000e+03,2.764000e+03,2.764000e+03
mean,9.044863e-08,2.532562e-08,-1.447178e-08,-2.170767e-08
std,6.536558e+00,4.870924e+00,2.912282e+00,2.568366e+00
min,-1.547531e+01,-1.452431e+01,-7.928900e+00,-8.329920e+00
25%,-5.099490e+00,-3.473265e+00,-1.972415e+00,-1.725280e+00
50%,-4.558100e-01,-2.075600e-01,-1.688400e-01,-6.886500e-02
75%,4.603195e+00,3.230165e+00,1.746265e+00,1.585002e+00
max,2.324204e+01,1.519141e+01,1.244072e+01,1.093867e+01


In [93]:
test4

array([[-0.78446454],
       [ 0.78446454],
       [ 1.17669681],
       [-1.17669681]])

In [80]:
same_traits = [0, 2]
diff_traits = [1, 3]

In [95]:
test = [-2.678498, 5.816443, -3.505566, -0.007615]
test2 = [0.087177, 0.925191, -1.526079, -3.750073]


In [61]:
# Example: recommend top 3 matches for each user
test = [-2.678498, 5.816443, -3.505566, -0.007615]


recs = recommend(test, data, same_traits, diff_traits, top_k=3)

In [100]:
np.array(data).flatten()

array([-1.09788112,  3.04265881,  0.44111017, ..., -1.57584742,
       -0.20959295, -0.2375467 ])